### Post-Success, Log Analytics:
- Do some analytics on logs.parquet.

In [ ]:
# ============================================================================
# CELL 1: Setup - Path Resolution & Imports
# ============================================================================

from pathlib import Path
import sys
import logging

# Suppress noisy logs for clean notebook output
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("finrag_ml_tg1").setLevel(logging.INFO)

# Find ModelPipeline root and add to sys.path
current = Path.cwd()
for parent in [current] + list(current.parents):
    if parent.name == "ModelPipeline":
        model_root = parent
        break
else:
    raise RuntimeError("Cannot find 'ModelPipeline' root in path tree")

if str(model_root) not in sys.path:
    sys.path.insert(0, str(model_root))

print(f"ModelPipeline root: {model_root}")
print(f"Notebook location: {Path.cwd()}")

ModelPipeline root: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline
Notebook location: d:\JoelDesktop folds_24\NEU FALL2025\MLops IE7374 Project\FinSights\ModelPipeline\finrag_ml_tg1\rag_modules_src\01_Isolation_Test_NBS


In [2]:
# ============================================================================
# DIAGNOSTIC: Inspect Query Logs Schema
# ============================================================================
import polars as pl
from pathlib import Path

# Path to logs
log_path = model_root / "finrag_ml_tg1" / "rag_modules_src" / "exports" / "logs" / "query_logs.parquet"

if not log_path.exists():
    print(f"ERROR: Log file not found at {log_path}")
else:
    # Load logs
    df_logs = pl.read_parquet(log_path)
    
    print(f"Total log entries: {df_logs.height}")
    print(f"Columns: {df_logs.columns}\n")
    
    # Show schema
    print("Schema:")
    print(df_logs.schema)
    print("\n" + "="*80 + "\n")
    
    # Show first 3 rows
    print("Sample rows:")
    print(df_logs.head(3))
    print("\n" + "="*80 + "\n")
    
    # Show data types and null counts
    print("Column stats:")
    for col in df_logs.columns:
        dtype = df_logs[col].dtype
        null_count = df_logs[col].null_count()
        print(f"  {col}: {dtype} (nulls: {null_count})")

Total log entries: 10
Columns: ['timestamp', 'query', 'model_id', 'input_tokens', 'output_tokens', 'total_tokens', 'cost', 'context_length', 'processing_time_ms', 'error', 'error_type', 'stage', 'context_file', 'response_file']

Schema:
Schema([('timestamp', String), ('query', String), ('model_id', String), ('input_tokens', Int64), ('output_tokens', Int64), ('total_tokens', Int64), ('cost', Float64), ('context_length', Int64), ('processing_time_ms', Float64), ('error', String), ('error_type', String), ('stage', String), ('context_file', String), ('response_file', String)])


Sample rows:
shape: (3, 14)
┌────────────┬────────────┬────────────┬───────────┬───┬───────────┬───────┬───────────┬───────────┐
│ timestamp  ┆ query      ┆ model_id   ┆ input_tok ┆ … ┆ error_typ ┆ stage ┆ context_f ┆ response_ │
│ ---        ┆ ---        ┆ ---        ┆ ens       ┆   ┆ e         ┆ ---   ┆ ile       ┆ file      │
│ str        ┆ str        ┆ str        ┆ ---       ┆   ┆ ---       ┆ str   ┆ ---       

In [3]:
# ============================================================================
# CELL: Section 1 - Overall Query History Summary
# ============================================================================
import polars as pl

# Load logs
log_path = model_root / "finrag_ml_tg1" / "rag_modules_src" / "exports" / "logs" / "query_logs.parquet"
df_logs = pl.read_parquet(log_path)

# Parse date from timestamp
df_logs = df_logs.with_columns([
    pl.col("timestamp").str.slice(0, 10).alias("date")
])

# Calculate summary stats
summary = pl.DataFrame({
    "Metric": [
        "Total Queries",
        "Date Range",
        "Unique Questions",
        "Total Cost",
        "Total Tokens",
        "Avg Cost per Query",
        "Avg Tokens per Query",
        "Avg Processing Time (ms)"
    ],
    "Value": [
        str(df_logs.height),
        f"{df_logs['date'].min()} to {df_logs['date'].max()}",
        str(df_logs["query"].n_unique()),
        f"${df_logs['cost'].sum():.4f}",
        f"{df_logs['total_tokens'].sum():,} ({df_logs['input_tokens'].sum():,} in + {df_logs['output_tokens'].sum():,} out)",
        f"${df_logs['cost'].mean():.4f}",
        f"{df_logs['total_tokens'].mean():,.0f}",
        f"{df_logs['processing_time_ms'].mean():,.1f}"
    ]
})

summary

Metric,Value
str,str
"""Total Queries""","""10"""
"""Date Range""","""2025-11-19 to 2025-11-19"""
"""Unique Questions""","""6"""
"""Total Cost""","""$0.1218"""
"""Total Tokens""","""74,178 (66,916 in + 7,262 out)"""
"""Avg Cost per Query""","""$0.0122"""
"""Avg Tokens per Query""","""7,418"""
"""Avg Processing Time (ms)""","""15,356.6"""


In [4]:
# ============================================================================
# CELL: Section 2 - Cost & Token Analysis by Query
# ============================================================================

# Truncate query text for readability
df_analysis = df_logs.with_columns([
    pl.when(pl.col("query").str.len_chars() > 80)
    .then(pl.col("query").str.slice(0, 80) + "...")
    .otherwise(pl.col("query"))
    .alias("query_short")
])

# Group by query
df_by_query = (
    df_analysis
    .group_by("query_short")
    .agg([
        pl.len().alias("runs"),
        pl.col("input_tokens").mean().round(0).cast(pl.Int64).alias("avg_input_tokens"),
        pl.col("output_tokens").mean().round(0).cast(pl.Int64).alias("avg_output_tokens"),
        pl.col("total_tokens").mean().round(0).cast(pl.Int64).alias("avg_total_tokens"),
        pl.col("cost").sum().alias("total_cost"),
        pl.col("cost").mean().alias("avg_cost"),
        pl.col("processing_time_ms").mean().round(1).alias("avg_processing_ms"),
    ])
    .sort("total_cost", descending=True)
)

df_by_query

query_short,runs,avg_input_tokens,avg_output_tokens,avg_total_tokens,total_cost,avg_cost,avg_processing_ms
str,u32,i64,i64,i64,f64,f64,f64
"""Across its fiscal 2018-2020 10…",5,5388,729,6117,0.063721,0.0127442,16326.3
"""Over time, how does Meta Platf…",1,11079,1270,12349,0.017429,0.017429,21283.6
"""For NVIDIA and Microsoft, what…",1,10607,1264,11871,0.016927,0.016927,20991.3
"""How does MICROSOFT CORP descri…",1,7066,365,7431,0.008891,0.008891,10087.1
"""Where does Tesla define Adjust…",1,5961,298,6259,0.007451,0.007451,8842.2
"""In their 2009 Form 10-K risk-f…",1,5263,418,5681,0.007353,0.007353,10729.8


In [5]:
# ============================================================================
# CELL: Section 3 - Model Usage Distribution
# ============================================================================

# Extract short model name
df_model_analysis = df_logs.with_columns([
    pl.col("model_id").str.split(".").list.last().alias("model_name")
])

# Group by model
df_by_model = (
    df_model_analysis
    .group_by("model_name")
    .agg([
        pl.len().alias("query_count"),
        pl.col("total_tokens").sum().alias("total_tokens"),
        pl.col("input_tokens").sum().alias("total_input_tokens"),
        pl.col("output_tokens").sum().alias("total_output_tokens"),
        pl.col("cost").sum().alias("total_cost"),
        pl.col("cost").mean().alias("avg_cost_per_query"),
        pl.col("processing_time_ms").mean().round(1).alias("avg_processing_ms"),
    ])
    .sort("query_count", descending=True)
)

df_by_model

model_name,query_count,total_tokens,total_input_tokens,total_output_tokens,total_cost,avg_cost_per_query,avg_processing_ms
str,u32,i64,i64,i64,f64,f64,f64
"""claude-haiku-4-5-20251001-v1:0""",9,68013,61528,6485,0.093953,0.010439,14911.0
"""claude-sonnet-4-5-20250929-v1:…",1,6165,5388,777,0.027819,0.027819,19366.9
